In [1]:
# -*-coding : utf-8 -*-

import numpy as np
import chainer
from chainer import cuda,Function,gradient_check,Variable,optimizers,serializers,utils
from chainer import Link,Chain,ChainList
import chainer.functions as F
import chainer.links as L

In [2]:
-

In [ ]:
import tqdm
import time 
import matplotlib.pyplot as plt
import argparse

parser = argparse.ArgumentParser(description='ChaosNN')
parser.add_argument('--batchsize', '-b', type=int, default=100,help='Number of images in each mini-batch')
parser.add_argument('--epoch', '-e', type=int, default=20,help='Number of sweeps over the dataset to train')
parser.add_argument('--gpu', '-g', type=int, default=-1,help='GPU ID (negative value indicates CPU)')
parser.add_argument('--out', '-o', default='result',help='Directory to output the result')
args = parser.parse_args()
print('GPU: {}'.format(args.gpu))
print('# Minibatch-size: {}'.format(args.batchsize))
print('# epoch: {}'.format(args.epoch))


model = ChaosNN()
if args.gpu >= 0:
    chainer.cuda.get_device(args.gpu).use() 
    model.to_gpu() 

optimizer = optimizers.Adam()
optimizer.setup(model)

N = len(train_x)
N_val = len(val_x)
batchsize = args.batchsize

train_acc = []
val_acc = []

for epoch in tqdm(xrange(1, args.epoch+1)):
    print 'epoch', epoch

    # training
    sum_accuracy = 0
    sum_loss = 0
    start = time.time()
    for i in xrange(0, N, batchsize):
        batch_x = train_x[i:i+batchsize]
        batch_y = train_y[i:i+batchsize]
        x0 = Variable(batch_x[0])
        x1 = Variable(batch_x[1])
        x2 = Variable(batch_x[2])
        x3 = Variable(batch_x[3])
        x4 = Variable(batch_x[4])
        y = Variable(batch_y)

        optimizer.update(model,x0,x1,x2,x3,x4,y)
        train_acc.append(float(model.accuracy.data))
        sum_loss += float(model.loss.data) * len(y.data)
        sum_accuracy += float(model.accuracy.data) * len(y.data)

    end = time.time()
    elapsed_time = end-start 
    throughput = N / elapsed_time

    print 'train mean loss={}, accuracy={}, throughput={} images/sec'.format(sum_loss / N, sum_accuracy / N, throughput)

    # evaluation
    sum_accuracy = 0
    sum_loss     = 0
    for i in xrange(0, N_val, batchsize):
        batch_x = val_x[i:i+batchsize]
        batch_y = val_y[i:i+batchsize]

        x0 = Variable(batch_x[0],volatile='on')
        x1 = Variable(batch_x[1],volatile='on')
        x2 = Variable(batch_x[2],volatile='on')
        x3 = Variable(batch_x[3],volatile='on')
        x4 = Variable(batch_x[4],volatile='on')
        y = Variable(batch_y,volatile='on')
        
        loss = model(x0,x1,x2,x3,x4,y)
        sum_loss += float(loss.data) * len(y.data)
        sum_accuracy += float(model.accuracy.data) * len(y.data)
    print 'test  mean loss={}, accuracy={}'.format(sum_loss / N_val, sum_accuracy / N_val)

    #Save
    outfile = args.out + str(epoch) + ".model"
    serializers.save_npz(outfile,model)

#Plot
plt.figure(figsize=(8,6))
plt.plot(range(len(train_acc)), train_acc)
plt.plot(range(len(val_acc)), val_acc)
plt.legend(["train_acc","val_acc"],loc=4)
plt.title("Accuracy of ChaosNN.")
plt.plot()
plt.savefig("plot1.png")
plt.show()